In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import seaborn as sns
from matplotlib import pyplot as plt

engine = create_engine('mssql+pyodbc://''@LAPTOP-JHB7LV8K\SQLEXPRESS/crops?driver=ODBC+Driver+17+for+SQL+Server')

query = 'SELECT * FROM products;'
df = pd.read_sql_query(query, engine)

#importing table from SQL as df

def read_query(query):
    result=pd.read_sql_query(query, engine)
    return result

#defining a function for further quieries and table for analysis

In [ ]:
df.shape
#checking for no of columns and rows

In [ ]:
df.columns
#checking for columns list

In [ ]:
df.head()
#returns first 5 rows

In [ ]:
df.info()

In [ ]:
#checking for null values
df.isnull().sum()
#no missing values exists in dataset
#no changes or modifications required in the dataset


In [ ]:
read_query("SELECT distinct([product group]) FROM products")

In [ ]:
read_query("SELECT count(distinct([product group])) FROM products")

In [ ]:
#Years on which data is to be visualised
read_query("SELECT distinct([Year]) FROM products")

In [ ]:
#checking products in product group
read_query("SELECT distinct([Product]) FROM products")

In [ ]:
#lets check for total quantity for 3 years
qty_year=read_query("SELECT sum(qty) as Qty_sum, year FROM products group by year ")
qty_year
qty_year.plot(kind='bar',x='year',y='Qty_sum',xlabel=('Year Range'),ylabel=('Total Quantity'),title=('Total Quantity by Year Range'))


In [ ]:
df.plot(kind='bar',x='year',y='Qty_sum')

In [ ]:
qty_prd_grp=read_query("SELECT distinct([product group]),sum(qty) as Qty_sum FROM products group by [product group]")
qty_prd_grp
list=qty_prd_grp['product group'].tolist()
qty_prd_grp.plot(kind='pie',y='Qty_sum',labels=list)
#above pie chart shows the quantity and product group and we can clearly see that cereals have a larger portion of contribution towards quantity,
#whereas cashew and floriculture are less contributrors

In [ ]:
#lets see the amount in USD generated by the export of products
prod_grp_qty_usd=read_query("select [PRODUCT],SUM(us$) as USD,year from Products group by  YEAR,[PRODUCT]")

prod_grp_qty_usd.plot(kind='bar',x='PRODUCT',y='USD',figsize=(20,7))

In [ ]:
#we have plotted graphs to see the different performance of product groups and products in regards to quantity
#now lets try to run stat functions on the below talble and try to draw some insights

qty_pred=read_query("with cte as (select sum(qty) as qty_sum,YEAR  from products  group by YEAR)select cte.qty_sum,year(RIGHT(year,4)) as year from cte")
qty_pred

In [ ]:
#gives the mean
abs(qty_pred.qty_sum.mean())


In [ ]:
#gives the median
qty_pred.qty_sum.median()

In [ ]:
#gives the range
qty_pred.qty_sum.max()-qty_pred.qty_sum.min()

In [ ]:
#gives the std deviation
import statistics
statistics.stdev(qty_pred.qty_sum)

In [ ]:
#gives the variance
statistics.variance(qty_pred.qty_sum)

In [ ]:
#gives the histogram representation
qty_pred.qty_sum.plot(kind='hist')


In [ ]:
#gives the bar graph
qty_pred.qty_sum.plot(kind='bar')

In [ ]:
#gives percetile of 75 %
import numpy
x = numpy.percentile(qty_pred.qty_sum,75)
x

In [ ]:
#gives the scatter plot of the table
X = qty_pred['year'].values.reshape(-1, 1)
y = qty_pred['qty_sum'].values
plt.scatter(x,y)

In [ ]:
#gives the correlation between the two variables 
Z=qty_pred.corr()
Z
#0.99 is a really good value and so we can run linear regression model to predict future values

In [ ]:
#Linear Regression to predict the future quantity
from sklearn.linear_model import LinearRegression

X = qty_pred['year'].values.reshape(-1, 1)#gives a list of values below one another
y = qty_pred['qty_sum'].values

# Create a linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict the qty_sum for 2022-2023
predicted_qty_sum = model.predict([[2022]])

print("Predicted qty_sum for 2022-2023:", predicted_qty_sum[0])


In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

y = qty_pred['year']
x = qty_pred['qty_sum']

slope, intercept, r, p, std_err = stats.linregress(x, y)

def myfunc(x):
  return slope * x + intercept

mymodel = [myfunc(x_val) for x_val in x]

plt.scatter(x, y)
plt.plot(x, mymodel)
plt.xlabel('qty_sum')
plt.ylabel('year')
plt.title('Linear Regression')
plt.show()
print(r)